# Semanlink automatic tagging and evaluation

This notebook present how to evaluate a neural search pipeline using pairs of query and answers. We will try to automatically tag arXiv papers that were manually automated by François-Paul Servant as part of the [Semanlink](http://www.semanlink.net/sl/home?lang=fr) Knowledge Graph.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pprint import pprint as print
from cherche import data, rank, retrieve, eval
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
documents, query_answers = data.arxiv_tags(arxiv_title=True, arxiv_summary=False, comment=False)

The `documents` contain a list of tags. Each tag is represented as a dict and contains a set of attributes. We will try to automate the tagging of arXiv documents with a neural search pipeline that will retrieve tags based on their attributes using the title, abstract and comments of the arXiv articles as a query.

This is what a pair of responses to a query looks like. For each query, there is a list of relevant document identifiers.

In [4]:
print(query_answers[:2])

[(' Joint Embedding of Words and Labels for Text Classification',
  [{'uri': 'http://www.semanlink.net/tag/deep_learning_attention'},
   {'uri': 'http://www.semanlink.net/tag/arxiv_doc'},
   {'uri': 'http://www.semanlink.net/tag/nlp_text_classification'},
   {'uri': 'http://www.semanlink.net/tag/label_embedding'}]),
 (' A Survey on Recent Approaches for Natural Language Processing in '
  'Low-Resource Scenarios',
  [{'uri': 'http://www.semanlink.net/tag/bosch'},
   {'uri': 'http://www.semanlink.net/tag/survey'},
   {'uri': 'http://www.semanlink.net/tag/arxiv_doc'},
   {'uri': 'http://www.semanlink.net/tag/nlp_low_resource_scenarios'},
   {'uri': 'http://www.semanlink.net/tag/low_resource_languages'}])]


Here is the list of attributes each tag has:

In [5]:
documents[0]

{'prefLabel': ['Attention mechanism'],
 'type': ['http://www.semanlink.net/2001/00/semanlink-schema#Tag'],
 'broader': ['http://www.semanlink.net/tag/deep_learning'],
 'creationTime': '2016-01-07T00:58:24Z',
 'creationDate': '2016-01-07',
 'comment': 'Good explanation is this [blog post by D. Britz](/doc/?uri=http%3A%2F%2Fwww.wildml.com%2F2016%2F01%2Fattention-and-memory-in-deep-learning-and-nlp%2F). (But the best explanation related to attention is to be found in this [post](/doc/2019/08/transformers_from_scratch_%7C_pet) about Self-Attention.) \r\n\r\nWhile simple Seq2Seq builds a single context vector out of the encoder’s last hidden state, attention creates\r\nshortcuts between the context vector and the entire source input: the context vector has access to the entire input sequence.\r\nThe decoder can “attend” to different parts of the source sentence at each step of the output generation, and the model learns what to attend to based on the input sentence and what it has produced 

Let's evaluate a first piepline made of a single retriever

In [25]:
from rapidfuzz import fuzz
from rapidfuzz import string_metric

fuzzers = [fuzz.ratio,
        fuzz.partial_ratio, fuzz.token_set_ratio, fuzz.partial_token_set_ratio,
        fuzz.token_sort_ratio, fuzz.partial_token_sort_ratio, fuzz.token_ratio,
        fuzz.partial_token_ratio, fuzz.WRatio, fuzz.QRatio, string_metric.levenshtein, string_metric.normalized_levenshtein]

for fuzzer in fuzzers:
    retriever = retrieve.Fuzz(key="uri", on = ["prefLabel_text",  "altLabel_text"], k=10, fuzzer=fuzzer)
    retriever.add(documents)
    print(eval.eval(search = retriever, query_answers=query_answers, hits_k=range(6))["R-Precision"])

'14.47%'
'18.91%'
'21.79%'
'10.75%'
'15.10%'
'12.79%'
'21.65%'
'10.75%'
'16.34%'
'14.47%'
'12.57%'
'13.49%'


In [8]:
retriever("bert")

[]

In [6]:
retriever = retrieve.TfIdf(
    key = "uri",
    on = ["prefLabel_text", "altLabel_text"], 
    documents = documents,
    tfidf = TfidfVectorizer(lowercase=True, min_df=1, max_df=0.9, ngram_range=(3, 7), analyzer="char"), 
    k = 10,
)

In [7]:
eval.eval(search = retriever, query_answers=query_answers, hits_k=range(6))

{'Precision@1': '63.06%',
 'Precision@2': '43.47%',
 'Precision@3': '33.12%',
 'Precision@4': '26.67%',
 'Precision@5': '22.55%',
 'Recall@1': '16.79%',
 'Recall@2': '22.22%',
 'Recall@3': '25.25%',
 'Recall@4': '27.03%',
 'Recall@5': '28.54%',
 'F1@1': '26.52%',
 'F1@2': '29.41%',
 'F1@3': '28.65%',
 'F1@4': '26.85%',
 'F1@5': '25.19%',
 'R-Precision': '26.95%',
 'Precision': '13.47%'}

In [8]:
retriever("ColBERTv2: Effective and Efficient Retrieval via Lightweight Late Interaction")

[{'uri': 'http://www.semanlink.net/tag/information_retrieval',
  'similarity': 0.2718},
 {'uri': 'http://www.semanlink.net/tag/retrieval_augmented_lm',
  'similarity': 0.21722},
 {'uri': 'http://www.semanlink.net/tag/dense_passage_retrieval',
  'similarity': 0.21111},
 {'uri': 'http://www.semanlink.net/tag/neural_models_for_information_retrieval',
  'similarity': 0.19562},
 {'uri': 'http://www.semanlink.net/tag/retrieval_based_nlp',
  'similarity': 0.18271},
 {'uri': 'http://www.semanlink.net/tag/ranking_information_retrieval',
  'similarity': 0.18235},
 {'uri': 'http://www.semanlink.net/tag/active_learning',
  'similarity': 0.15662},
 {'uri': 'http://www.semanlink.net/tag/embeddings_in_ir',
  'similarity': 0.1552},
 {'uri': 'http://www.semanlink.net/tag/entity_discovery_and_linking',
  'similarity': 0.1393},
 {'uri': 'http://www.semanlink.net/tag/information_extraction',
  'similarity': 0.13636}]

The results of Lunr are inferior to TfIdf on this dataset.

In [9]:
retriever = retrieve.Lunr(
    key = "uri",
    on = ["prefLabel_text", "altLabel_text"], 
    documents = documents,
    k = 10,
)

eval.eval(search = retriever, query_answers=query_answers, hits_k=range(6))

{'Precision@1': '56.55%',
 'Precision@2': '42.10%',
 'Precision@3': '34.17%',
 'Precision@4': '27.80%',
 'Precision@5': '23.33%',
 'Recall@1': '14.90%',
 'Recall@2': '21.52%',
 'Recall@3': '25.86%',
 'Recall@4': '27.74%',
 'Recall@5': '28.59%',
 'F1@1': '23.59%',
 'F1@2': '28.48%',
 'F1@3': '29.44%',
 'F1@4': '27.77%',
 'F1@5': '25.69%',
 'R-Precision': '27.59%',
 'Precision': '14.00%'}

You can find an explanation of the metrics [here](https://amitness.com/2020/08/information-retrieval-evaluation/). The TfIdf retriever using caracters ngrams did well.

Here is what tagging looks like using our retriever

In [10]:
retriever("ColBERTv2: Effective and Efficient Retrieval via Lightweight Late Interaction")

[{'uri': 'http://www.semanlink.net/tag/information_retrieval',
  'similarity': 4.147},
 {'uri': 'http://www.semanlink.net/tag/dense_passage_retrieval',
  'similarity': 3.489},
 {'uri': 'http://www.semanlink.net/tag/ranking_information_retrieval',
  'similarity': 3.489},
 {'uri': 'http://www.semanlink.net/tag/embeddings_in_ir', 'similarity': 3.489},
 {'uri': 'http://www.semanlink.net/tag/retrieval_augmented_lm',
  'similarity': 3.489},
 {'uri': 'http://www.semanlink.net/tag/retrieval_based_nlp',
  'similarity': 3.489},
 {'uri': 'http://www.semanlink.net/tag/entity_discovery_and_linking',
  'similarity': 1.579},
 {'uri': 'http://www.semanlink.net/tag/neural_models_for_information_retrieval',
  'similarity': 1.479}]

Let's try to improve those results using a ranker.

In [11]:
retriever = retrieve.TfIdf(
    key = "uri",
    on = ["prefLabel_text", "altLabel_text"], 
    documents = documents,
    tfidf = TfidfVectorizer(lowercase=True, min_df=1, max_df=0.9, ngram_range=(3, 7), analyzer="char"), 
    k = 30,
)

ranker = rank.Encoder(
    key = "uri",
    on = ["prefLabel_text", "altLabel_text"],
    encoder = SentenceTransformer("sentence-transformers/all-mpnet-base-v2").encode,
    k = 10,
    path = "semanlink.pkl"
)

The cell below is important if you want to pre-compute the queries once. 314 queries are pre-calculated to speed up evaluation. It takes roughly CPU 24 seconds.

In [12]:
ranker.add([q for q, _ in query_answers])

Encoder ranker
	 key: uri
	 on: prefLabel_text, altLabel_text
	 k: 10
	 similarity: cosine
	 embeddings stored at: semanlink.pkl

In [13]:
search = retriever + ranker
search.add(documents)

TfIdf retriever
 	 key: uri
 	 on: prefLabel_text, altLabel_text
 	 documents: 433
Encoder ranker
	 key: uri
	 on: prefLabel_text, altLabel_text
	 k: 10
	 similarity: cosine
	 embeddings stored at: semanlink.pkl

In [14]:
eval.eval(search = search, query_answers = query_answers, hits_k=range(6))

{'Precision@1': '63.69%',
 'Precision@2': '42.83%',
 'Precision@3': '33.01%',
 'Precision@4': '26.75%',
 'Precision@5': '22.55%',
 'Recall@1': '17.15%',
 'Recall@2': '22.64%',
 'Recall@3': '25.75%',
 'Recall@4': '27.34%',
 'Recall@5': '28.55%',
 'F1@1': '27.02%',
 'F1@2': '29.63%',
 'F1@3': '28.93%',
 'F1@4': '27.04%',
 'F1@5': '25.20%',
 'R-Precision': '27.43%',
 'Precision': '13.18%'}

The Bert Sentence classifier improved the results of the extractor a little. We managed to increase the F1@k score, precision and recall.

Here are proposed tags for Bert using retriever ranker:

In [15]:
search("ColBERTv2: Effective and Efficient Retrieval via Lightweight Late Interaction")

[{'uri': 'http://www.semanlink.net/tag/retrieval_augmented_lm',
  'similarity': 0.5449115},
 {'uri': 'http://www.semanlink.net/tag/neural_models_for_information_retrieval',
  'similarity': 0.42808777},
 {'uri': 'http://www.semanlink.net/tag/embeddings_in_ir',
  'similarity': 0.42719522},
 {'uri': 'http://www.semanlink.net/tag/dense_passage_retrieval',
  'similarity': 0.42641854},
 {'uri': 'http://www.semanlink.net/tag/information_retrieval',
  'similarity': 0.40513232},
 {'uri': 'http://www.semanlink.net/tag/entity_discovery_and_linking',
  'similarity': 0.352889},
 {'uri': 'http://www.semanlink.net/tag/ranking_information_retrieval',
  'similarity': 0.346283},
 {'uri': 'http://www.semanlink.net/tag/retrieval_based_nlp',
  'similarity': 0.32937074},
 {'uri': 'http://www.semanlink.net/tag/active_learning',
  'similarity': 0.28093448},
 {'uri': 'http://www.semanlink.net/tag/cognitive_search',
  'similarity': 0.25938064}]

Let's try to use using Flash as a retriever. Flash Text will retrieve tags labels inside the title. 

In [16]:
retriever = retrieve.Flash(
    key = "uri",
    on = ["prefLabel", "altLabel"], 
    k = 30,
)

search = retriever + ranker
search.add(documents)

Flash retriever
 	 key: uri
 	 on: prefLabel, altLabel
 	 documents: 605
Encoder ranker
	 key: uri
	 on: prefLabel_text, altLabel_text
	 k: 10
	 similarity: cosine
	 embeddings stored at: semanlink.pkl

FlashText as a retriever provides fewer candidates than TfIdf but has higher precision.

In [17]:
eval.eval(search = search, query_answers = query_answers, hits_k=range(6))

{'Precision@1': '72.80%',
 'Precision@2': '61.90%',
 'Precision@3': '59.90%',
 'Precision@4': '59.27%',
 'Precision@5': '59.37%',
 'Recall@1': '16.33%',
 'Recall@2': '19.54%',
 'Recall@3': '20.11%',
 'Recall@4': '20.16%',
 'Recall@5': '20.20%',
 'F1@1': '26.67%',
 'F1@2': '29.71%',
 'F1@3': '30.11%',
 'F1@4': '30.08%',
 'F1@5': '30.15%',
 'R-Precision': '20.20%',
 'Precision': '59.37%'}

In [18]:
search("ColBERTv2: Effective and Efficient Retrieval via Lightweight Late Interaction")

[]

We can get the best of both worlds by using pipeline union. It gets a bit complicated, but the union allows us to retrieve the best candidates from the first model, then add the candidates from the second model without duplicates (etc, no matter how many models are in the union). Our first retriever and ranker (Flash + Encoder) has low recall and high precision. The second retriever has lower precision but higher recall. We can mix things up and offer Flash and Ranker candidates first, then TfIdf and Ranker candidates seconds.

In [19]:
precision = retrieve.Flash(
    key = "uri",
    on = ["prefLabel", "altLabel"], 
    k = 30,
) + ranker

recall = retrieve.TfIdf(
    key = "uri",
    on = ["prefLabel_text", "altLabel_text"], 
    documents = documents,
    tfidf = TfidfVectorizer(lowercase=True, min_df=1, max_df=0.9, ngram_range=(3, 7), analyzer="char"), 
    k = 30,
) + ranker

search = precision | recall

search.add(documents)

Union Pipeline
-----
Flash retriever
 	 key: uri
 	 on: prefLabel, altLabel
 	 documents: 605
Encoder ranker
	 key: uri
	 on: prefLabel_text, altLabel_text
	 k: 10
	 similarity: cosine
	 embeddings stored at: semanlink.pkl
TfIdf retriever
 	 key: uri
 	 on: prefLabel_text, altLabel_text
 	 documents: 433
Encoder ranker
	 key: uri
	 on: prefLabel_text, altLabel_text
	 k: 10
	 similarity: cosine
	 embeddings stored at: semanlink.pkl
-----

In [20]:
eval.eval(search = search, query_answers = query_answers, hits_k=range(6))

{'Precision@1': '69.11%',
 'Precision@2': '49.84%',
 'Precision@3': '39.07%',
 'Precision@4': '31.13%',
 'Precision@5': '25.92%',
 'Recall@1': '18.74%',
 'Recall@2': '25.89%',
 'Recall@3': '30.10%',
 'Recall@4': '31.58%',
 'Recall@5': '32.57%',
 'F1@1': '29.49%',
 'F1@2': '34.08%',
 'F1@3': '34.00%',
 'F1@4': '31.35%',
 'F1@5': '28.87%',
 'R-Precision': '31.96%',
 'Precision': '13.97%'}

We did improves F1 and recall scores using union of pipelines.

Here are our tags for BERT's article with best of both worlds

In [21]:
search("ColBERTv2: Effective and Efficient Retrieval via Lightweight Late Interaction")

[{'uri': 'http://www.semanlink.net/tag/retrieval_augmented_lm'},
 {'uri': 'http://www.semanlink.net/tag/neural_models_for_information_retrieval'},
 {'uri': 'http://www.semanlink.net/tag/embeddings_in_ir'},
 {'uri': 'http://www.semanlink.net/tag/dense_passage_retrieval'},
 {'uri': 'http://www.semanlink.net/tag/information_retrieval'},
 {'uri': 'http://www.semanlink.net/tag/entity_discovery_and_linking'},
 {'uri': 'http://www.semanlink.net/tag/ranking_information_retrieval'},
 {'uri': 'http://www.semanlink.net/tag/retrieval_based_nlp'},
 {'uri': 'http://www.semanlink.net/tag/active_learning'},
 {'uri': 'http://www.semanlink.net/tag/cognitive_search'}]

Another example of tagging:

In [22]:
search("Knowledge Base Embedding By Cooperative Knowledge Distillation")

[{'uri': 'http://www.semanlink.net/tag/knowledge_base'},
 {'uri': 'http://www.semanlink.net/tag/knowledge_distillation'},
 {'uri': 'http://www.semanlink.net/tag/embeddings'},
 {'uri': 'http://www.semanlink.net/tag/knowledge_graph_embeddings'},
 {'uri': 'http://www.semanlink.net/tag/knowledge_driven_embeddings'},
 {'uri': 'http://www.semanlink.net/tag/hierarchy_aware_knowledge_graph_embeddings'},
 {'uri': 'http://www.semanlink.net/tag/text_kg_and_embeddings'},
 {'uri': 'http://www.semanlink.net/tag/text_aware_kg_embedding'},
 {'uri': 'http://www.semanlink.net/tag/knowledge_graph_completion'},
 {'uri': 'http://www.semanlink.net/tag/knowledge_graph_deep_learning'},
 {'uri': 'http://www.semanlink.net/tag/combining_knowledge_graphs'},
 {'uri': 'http://www.semanlink.net/tag/multiple_knowledge_bases'}]

### CKB 

In [179]:
import torch
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
model = torch.load("mpnet-base-v2-semanlink")

tokenizer.save_pretrained("semanlink_model")
model.model.save_pretrained("semanlink_model")

In [183]:
retriever = retrieve.Encoder(
    encoder = SentenceTransformer("semanlink_model").encode,
    key = "uri",
    on = ["prefLabel_text", "altLabel_text"],
    k = 30,
    path = "test.pkl",
)

retriever.add(documents)
retriever.add([q for q, _ in query_answers])

WARNING -  No sentence-transformers model found with name semanlink_model. Creating a new one with MEAN pooling.


Encoder retriever
 	 key: uri
 	 on: prefLabel_text, altLabel_text
 	 documents: 433

In [184]:
eval.eval(search = retriever, query_answers = query_answers, hits_k=range(6))

{'Precision@1': '35.67%',
 'Precision@2': '25.00%',
 'Precision@3': '19.96%',
 'Precision@4': '17.12%',
 'Precision@5': '14.71%',
 'Recall@1': '9.84%',
 'Recall@2': '13.41%',
 'Recall@3': '16.00%',
 'Recall@4': '18.08%',
 'Recall@5': '19.51%',
 'F1@1': '15.42%',
 'F1@2': '17.45%',
 'F1@3': '17.76%',
 'F1@4': '17.59%',
 'F1@5': '16.78%',
 'R-Precision': '17.68%',
 'Precision': '4.59%'}

In [178]:
eval.eval(search = retriever, query_answers = query_answers, hits_k=range(6))

{'Precision@1': '35.67%',
 'Precision@2': '25.00%',
 'Precision@3': '19.96%',
 'Precision@4': '17.12%',
 'Precision@5': '14.71%',
 'Recall@1': '9.84%',
 'Recall@2': '13.41%',
 'Recall@3': '16.00%',
 'Recall@4': '18.08%',
 'Recall@5': '19.51%',
 'F1@1': '15.42%',
 'F1@2': '17.45%',
 'F1@3': '17.76%',
 'F1@4': '17.59%',
 'F1@5': '16.78%',
 'R-Precision': '17.68%',
 'Precision': '4.59%'}

In [80]:
retriever("max")

[{'uri': 'http://www.semanlink.net/tag/max_halford',
  'similarity': 12.090814707875586},
 {'uri': 'http://www.semanlink.net/tag/cold_start_problem',
  'similarity': 10.307108815484117},
 {'uri': 'http://www.semanlink.net/tag/lime', 'similarity': 9.84508999915719},
 {'uri': 'http://www.semanlink.net/tag/ernie',
  'similarity': 9.762842843646684},
 {'uri': 'http://www.semanlink.net/tag/nlp_long_documents',
  'similarity': 9.419577852318142},
 {'uri': 'http://www.semanlink.net/tag/good', 'similarity': 9.383351128118539},
 {'uri': 'http://www.semanlink.net/tag/knowbert',
  'similarity': 9.202494437922873},
 {'uri': 'http://www.semanlink.net/tag/oov', 'similarity': 9.11512814978782},
 {'uri': 'http://www.semanlink.net/tag/surprises_me',
  'similarity': 9.049567481741605},
 {'uri': 'http://www.semanlink.net/tag/starspace',
  'similarity': 8.966195161873559},
 {'uri': 'http://www.semanlink.net/tag/xlnet',
  'similarity': 8.860616356016111},
 {'uri': 'http://www.semanlink.net/tag/zero_shot',


#### Sentence BERT vanilla

In [62]:
retriever = retrieve.Encoder(
    key = "uri",
    on = ["prefLabel_text", "altLabel_text"],
    encoder = SentenceTransformer("sentence-transformers/all-mpnet-base-v2").encode,
    k = 10,
    path = "semanlink.pkl"
)

retriever.add(documents)
retriever.add([q for q, _ in query_answers])

Encoder retriever
 	 key: uri
 	 on: prefLabel_text, altLabel_text
 	 documents: 433

In [63]:
eval.eval(search = retriever, query_answers = query_answers, hits_k=range(6))

{'Precision@1': '61.46%',
 'Precision@2': '41.40%',
 'Precision@3': '31.32%',
 'Precision@4': '25.56%',
 'Precision@5': '21.85%',
 'Recall@1': '16.71%',
 'Recall@2': '21.94%',
 'Recall@3': '24.54%',
 'Recall@4': '26.45%',
 'Recall@5': '27.88%',
 'F1@1': '26.28%',
 'F1@2': '28.68%',
 'F1@3': '27.52%',
 'F1@4': '26.00%',
 'F1@5': '24.50%',
 'R-Precision': '26.78%',
 'Precision': '13.25%'}

### Pipeline

In [44]:
precision = retrieve.Flash(
    key = "uri",
    on = ["prefLabel", "altLabel"], 
    k = 30,
) + ranker

recall = retrieve.TfIdf(
    key = "uri",
    on = ["prefLabel_text", "altLabel_text"], 
    documents = documents,
    tfidf = TfidfVectorizer(lowercase=True, min_df=1, max_df=0.9, ngram_range=(3, 7), analyzer="char"), 
    k = 30,
) + ranker

search = precision | recall

search.add(documents)

Union Pipeline
-----
Flash retriever
 	 key: uri
 	 on: prefLabel, altLabel
 	 documents: 605
Encoder ranker
	 key: uri
	 on: prefLabel_text, altLabel_text
	 k: 10
	 similarity: cosine
	 embeddings stored at: semanlink_custom.pkl
TfIdf retriever
 	 key: uri
 	 on: prefLabel_text, altLabel_text
 	 documents: 433
Encoder ranker
	 key: uri
	 on: prefLabel_text, altLabel_text
	 k: 10
	 similarity: cosine
	 embeddings stored at: semanlink_custom.pkl
-----

In [45]:
eval.eval(search = search, query_answers = query_answers, hits_k=range(6))

{'Precision@1': '64.65%',
 'Precision@2': '46.34%',
 'Precision@3': '34.71%',
 'Precision@4': '27.31%',
 'Precision@5': '22.99%',
 'Recall@1': '17.24%',
 'Recall@2': '24.35%',
 'Recall@3': '27.07%',
 'Recall@4': '28.24%',
 'Recall@5': '29.42%',
 'F1@1': '27.21%',
 'F1@2': '31.92%',
 'F1@3': '30.42%',
 'F1@4': '27.77%',
 'F1@5': '25.81%',
 'R-Precision': '28.72%',
 'Precision': '12.97%'}

In [ ]:
retr